In [1]:
import innvestigate
import tensorflow as tf
from pathlib import Path
import xarray as xr
from shared.training import split_data_on_participants, train_and_evaluate
from shared.normalization import *
from shared.generators import SAT1DataGenerator
import matplotlib.pyplot as plt
import numpy as np
from shared.data import SAT1_STAGES_ACCURACY
from keras.models import load_model

# tf.compat.v1.disable_eager_execution()
from mne.viz import plot_topomap
from mne.io import read_info
from collections import defaultdict
from shared.lstm_network import LSTM_network

2023-10-09 15:17:14.149959: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 15:17:14.727740: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

train_data, val_data, test_data = split_data_on_participants(data, 60, norm_dummy)

In [3]:
positions = read_info(Path("data/sat1/preprocessed/processed_0001_epo.fif"))

In [10]:
keras_model = load_model('models/sentiment_model.hdf5')

In [4]:
model = tf.keras.models.load_model("models/gru")

n_hidden = 256
embedding_dim = 30
n_classes = 5
weights = model.get_weights()
print(len(weights))

2023-10-09 15:17:22.679979: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-09 15:17:22.709455: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-09 15:17:22.709529: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-09 15:17:22.711716: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-09 15:17:22.711785: I tensorflow/compile

7


In [5]:
for weight in weights:
    print(weight.shape)

(30, 768)
(256, 768)
(2, 768)
(256, 128)
(128,)
(128, 5)
(5,)


In [5]:
weights[0].shape

(30, 768)

In [6]:
weights[3].shape

(256, 5)

In [7]:
weights.append(np.zeros((n_classes,)))
lrp_model = LSTM_network(n_hidden, embedding_dim, n_classes, weights=weights)

AssertionError: 